In [1]:
import pandas as pd

In [2]:
rules = pd.read_json("data/arules.json")
rules["antecedents"] = rules["antecedents"].apply(lambda x: frozenset(x))
rules["consequents"] = rules["consequents"].apply(lambda x: frozenset(x))

In [3]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(IAmA),(AskReddit),0.217120,0.480390,0.127371,0.586641,1.221177,0.023069,1.257043
1,(fffffffuuuuuuuuuuuu),(IAmA),0.003648,0.217120,0.002052,0.562500,2.590737,0.001260,1.789441
2,(todayilearned),(IAmA),0.013818,0.217120,0.008072,0.584158,2.690491,0.005072,1.882641
3,(wikipedia),(IAmA),0.004378,0.217120,0.002280,0.520833,2.398831,0.001330,1.633837
4,(bestof),(IAmA),0.002782,0.217120,0.001687,0.606557,2.793655,0.001083,1.989821
...,...,...,...,...,...,...,...,...,...
102302,"(science, pics, worldnews, politics, programmi...","(reddit.com, WTF, technology)",0.002417,0.017968,0.001323,0.547170,30.452639,0.001279,2.168654
102303,"(science, worldnews, politics, programming, WT...","(reddit.com, technology, pics)",0.002371,0.017785,0.001323,0.557692,31.356607,0.001280,2.220659
102304,"(pics, worldnews, politics, programming, WTF, ...","(science, reddit.com, technology)",0.002645,0.014867,0.001323,0.500000,33.631902,0.001283,1.970266
102305,"(worldnews, technology, politics, programming,...","(science, WTF, reddit.com, pics)",0.002326,0.021707,0.001323,0.568627,26.195090,0.001272,2.267860


In [4]:
import sqlite3
from itertools import chain

In [5]:
con = sqlite3.connect('data/example.db')

In [6]:
def create_db(conn):
    cur = con.cursor()
    cur.execute('''
    CREATE TABLE IF NOT EXISTS Subreddits (
        id integer PRIMARY KEY,
        name text NOT NULL
    )
    ''')

    cur.execute('''
    CREATE TABLE IF NOT EXISTS Rules (
        id INTEGER PRIMARY KEY,
        support REAL NOT NULL DEFAULT 0,
        confidence REAL NOT NULL DEFAULT 0,
        lift REAL NOT NULL DEFAULT 0
    )
    ''')


    cur.execute('''
    CREATE TABLE IF NOT EXISTS Antecedents (
        rule_id integer not null,
        subreddit_id integer not null,
        FOREIGN KEY(subreddit_id) REFERENCES Subredsits(id),
        FOREIGN KEY(rule_id) REFERENCES Rules(id),
        PRIMARY KEY(rule_id, subreddit_id)
    )
    ''')

    cur.execute('''
    CREATE TABLE IF NOT EXISTS Consequents (
        rule_id integer not null,
        subreddit_id integer not null,
        FOREIGN KEY(subreddit_id) REFERENCES Subredsits(id),
        FOREIGN KEY(rule_id) REFERENCES Rules(id),
        PRIMARY KEY(rule_id, subreddit_id)
    )
    ''')
    con.commit()

In [7]:
def add_rule(rule, cur):
    data = (rule["support"], rule["confidence"], rule["lift"])
    
    sql = '''INSERT INTO Rules (support,confidence, lift) VALUES(?,?,?)'''
    cur.execute(sql,data)
    return cur.lastrowid

def add_antecedents(rule_id, antecedents, cur):    
    sql = '''INSERT INTO antecedents(rule_id, subreddit_id) VALUES '''
    sql += ','.join(["(?, (SELECT id from Subreddits WHERE name = ?))" for i in range(len(antecedents))])
    data = chain.from_iterable(zip([rule_id]*len(antecedents), [an for an in antecedents]))
    cur.execute(sql, tuple(data))
    
def add_consequents(rule_id, consequents, cur):    
    sql = '''INSERT INTO Consequents(rule_id, subreddit_id) VALUES '''
    sql += ','.join(["(?, (SELECT id from Subreddits WHERE name = ?))" for i in range(len(consequents))])
    data = chain.from_iterable(zip([rule_id]*len(consequents), [an for an in consequents]))
    cur.execute(sql, tuple(data))
    
    
def add_subreddits(subreddits, cur):
    sql = '''SELECT name FROM Subreddits'''
    cur.execute(sql)
    already_added = set([sub[0] for sub in cur.fetchall()])
    to_add = subreddits - already_added
    if len(to_add) == 0:
        return
    sql = 'INSERT INTO subreddits(name) values '+','.join([ "(?)" for i in range(len(to_add))])
    data = list(to_add)
    cur.execute(sql,data)
        

In [8]:
create_db(con)

In [9]:
subreddits = set(item for sublist in rules["antecedents"] for item in sublist)

In [10]:
cur=con.cursor()
add_subreddits(subreddits, cur)

In [11]:
cur = con.cursor()
for index, row in rules.iterrows():
    rule_id = add_rule(row,cur)
    add_antecedents(rule_id, row["antecedents"],cur)
    add_consequents(rule_id, row["consequents"], cur)
    
con.commit()

In [12]:
def get_antecendents(rule_id, cur):
    sql = '''
    SELECT subreddits.name
    FROM rules
    INNER JOIN antecedents ON rules.id = Antecedents.rule_id
    INNER JOIN subreddits ON Antecedents.subreddit_id = subreddits.id
    WHERE rules.id = ?
    '''
    data = (rule_id,)
    cur.execute(sql,data)
    antecedents = cur.fetchall()
    return antecedents

def get_consequents(rule_id, cur):
    sql = '''
    SELECT subreddits.name
    FROM rules
    INNER JOIN Consequents ON rules.id = Consequents.rule_id
    INNER JOIN subreddits ON Consequents.subreddit_id = subreddits.id
    WHERE rules.id = ?
    '''
    data = (rule_id,)
    cur.execute(sql,data)
    consequents = cur.fetchall()
    return consequents

def get_rule_data(rule_id, cur):
    sql = '''
    SELECT *
    FROM rules
    WHERE rules.id=?'''
    data=(rule_id,)
    cur.execute(sql,data)
    res = cur.fetchall()
    if len(res) == 0:
        return None
    return res[0]

def get_rule(rule_id,cur):
    rule = get_rule_data(rule_id, cur)
    antecedents = get_antecendents(rule_id, cur)
    consequents = get_consequents(rule_id, cur)
    print(rule)
    print("IF:",antecedents)
    print("THEN",consequents)


In [13]:
get_rule(95, cur)

(95, 0.0015505290000000001, 0.5074626866, 1.7674145157)
IF: [('bicycling',)]
THEN [('reddit.com',)]


In [74]:
def get_subreddit_ids(subreddits, cur):
    sql = 'SELECT id FROM Subreddits WHERE name IN ('+','.join(["?"]*len(subreddits))+")"
    data = tuple(subreddits)
    cur.execute(sql, data)
    result = set([sub[0] for sub in cur.fetchall()])
    return result

def get_appropriate_rule_ids(subreddit_ids, cur):
    sql = '''
    SELECT R.id
    FROM rules as R
    INNER JOIN antecedents as A ON R.id = A.rule_id
    INNER JOIN Consequents as C ON R.id = C.rule_id
    WHERE 
    C.subreddit_id NOT IN ('''+','.join(["?"]*len(subreddit_ids))+''')
    GROUP BY R.id
    HAVING
    count(*) > 0 AND
    count(*) = (SELECT count(*)
                FROM antecedents 
                WHERE rule_id = R.id AND
                antecedents.subreddit_id IN ('''+','.join(["?"]*len(subreddit_ids))+'''))
    '''
    
    cur.execute(sql, tuple(subreddit_ids)+tuple(subreddit_ids))
    rule_candidates = [el[0] for el in cur.fetchall()]
    return rule_candidates


def get_recomendations(subreddits, cur):
    subreddit_ids = get_subreddit_ids(subreddits, cur)
    rule_candidates = get_appropriate_rule_ids(subreddit_ids, cur)
    sql = '''
    SELECT S.name
    FROM consequents AS C
    INNER JOIN Rules as R ON C.rule_id = R.id
    INNER JOIN subreddits as S ON C.subreddit_id = S.id
    WHERE 
    C.rule_id IN ('''+','.join(["?"]*len(rule_candidates))+''') AND
    C.subreddit_id NOT IN ('''+','.join(["?"]*len(subreddit_ids))+''')
    GROUP BY C.subreddit_id
    ORDER BY MAX(R.confidence + R.support) DESC
    '''
    cur.execute(sql, tuple(rule_candidates)+ tuple(subreddit_ids))
    return cur.fetchall()
    

In [75]:
get_subreddit_ids({'funny', 'WTF', 'math', 'fffffffuuuuuuuuuuuu', 'bestof'}, cur)

{13, 27, 37, 40, 67}

In [76]:
ids = get_appropriate_rule_ids(get_subreddit_ids({'funny', 'WTF', 'math', 'fffffffuuuuuuuuuuuu', 'bestof'},cur), cur)
for idx in ids:
    get_rule(idx, cur)

(2, 0.0020521707, 0.5625, 2.5907372401)
IF: [('fffffffuuuuuuuuuuuu',)]
THEN [('IAmA',)]
(5, 0.0016873404, 0.606557377, 2.7936547288)
IF: [('bestof',)]
THEN [('IAmA',)]
(13, 0.0024626049, 0.675, 1.4051072717)
IF: [('fffffffuuuuuuuuuuuu',)]
THEN [('AskReddit',)]
(14, 0.07688799710000001, 0.5785861359000001, 1.2044082768)
IF: [('WTF',)]
THEN [('AskReddit',)]
(31, 0.0021889821, 0.7868852459, 1.63801212)
IF: [('bestof',)]
THEN [('AskReddit',)]
(33, 0.046196643600000004, 0.5872463768, 1.2224357842)
IF: [('funny',)]
THEN [('AskReddit',)]
(59, 0.0020521707, 0.5625, 1.9591010165)
IF: [('fffffffuuuuuuuuuuuu',)]
THEN [('reddit.com',)]
(60, 0.0018697556, 0.5125000000000001, 3.2583647434)
IF: [('fffffffuuuuuuuuuuuu',)]
THEN [('pics',)]
(62, 0.08546151040000001, 0.6431022649, 2.2398263128)
IF: [('WTF',)]
THEN [('reddit.com',)]
(63, 0.0666271434, 0.5013726836, 3.1876196596)
IF: [('WTF',)]
THEN [('pics',)]
(93, 0.0020521707, 0.737704918, 2.5693128085)
IF: [('bestof',)]
THEN [('reddit.com',)]
(94, 0.05

In [77]:
# for i in range(1200):
#     get_rule(i, cur)

In [78]:
%timeit -n 3 get_recomendations({'funny', 'WTF', 'math', 'fffffffuuuuuuuuuuuu', 'bestof', 'offbeat'}, cur)

422 ms ± 5.79 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [81]:
get_recomendations({'funny', 'WTF', 'math', 'fffffffuuuuuuuuuuuu', 'bestof', 'offbeat'}, cur)

[('pics',),
 ('reddit.com',),
 ('AskReddit',),
 ('IAmA',),
 ('programming',),
 ('science',),
 ('politics',)]

In [44]:
cur.execute('''SELECT count(*) from antecedents''')
cur.fetchall()

[(440406,)]

In [19]:
#better sql
sql = '''
    SELECT R.id
    FROM rules as R
    INNER JOIN antecedents as A ON R.id = A.rule_id
    INNER JOIN Consequents as C ON R.id = C.rule_id
    WHERE 
    C.subreddit_id NOT IN ('''+','.join(["?"]*len(subreddit_ids))+''')
    GROUP BY R.id
    HAVING 
    A.subreddit_id ALL IN ('''+','.join(["?"]*len(subreddit_ids))+''')
    '''

NameError: name 'subreddit_ids' is not defined

In [ ]:
sql = '''
    SELECT R.id
    FROM rules as R
    INNER JOIN antecedents as A ON R.id = A.rule_id
    INNER JOIN Consequents as C ON R.id = C.rule_id
    INNER JOIN subreddits as SA on A.subreddit_id = SA.id
    INNER JOIN subreddits as SC on C.subreddit_id = SC.id
    GROUP BY R.id
    HAVING 
    ALL A.subreddit_id
    '''